In [ ]:
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import re

# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Definir o caminho principal do dataset
dataset_path = '/content/drive/My Drive/vision_uerj'  # Certifique-se de que este caminho está correto

# Função para carregar imagens e metadados a partir do nome do arquivo
def load_dataset_from_filenames(dataset_path):
    data = []
    
    # Percorrer cada pasta da classe (Caneca, Chave, etc.)
    for class_folder in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_folder)
        
        if os.path.isdir(class_path):
            # Percorrer as subpastas (dia_dentro, dia_fora, noite_dentro, noite_fora)
            for subfolder in os.listdir(class_path):
                subfolder_path = os.path.join(class_path, subfolder)
                
                if os.path.isdir(subfolder_path):
                    # Agora procurar as imagens dentro dessas subpastas
                    for img_file in os.listdir(subfolder_path):
                        if img_file.endswith('.HEIC') or img_file.endswith('.jpg'):  # Verificar tipos de arquivo
                            # Extrair os metadados do nome do arquivo
                            parts = re.split('[_-]', img_file.replace('.HEIC', '').replace('.jpg', ''))
                            
                            # Verificar se o nome do arquivo tem as partes esperadas (classe, objeto, fundo, iluminação, textura)
                            if len(parts) == 5:
                                classe = parts[0]  # Classe do objeto (ex: Caneca, Chave, etc.)
                                numero_objeto = parts[1]  # Número do objeto (ex: 1, 2, 3...)
                                fundo = parts[2]  # Fundo (ex: dentro, fora)
                                iluminacao = parts[3]  # Iluminação (ex: dia, noite)
                                textura = parts[4]  # Número da textura (ex: 1, 2, 3...)

                                # Caminho completo da imagem
                                img_path = os.path.join(subfolder_path, img_file)

                                # Adicionar os dados em uma lista
                                data.append({
                                    'nome_arquivo': img_file,
                                    'classe': classe,
                                    'numero_objeto': numero_objeto,
                                    'fundo': fundo,
                                    'iluminacao': iluminacao,
                                    'textura': textura,
                                    'caminho': img_path
                                })
                            else:
                                print(f"Aviso: O arquivo '{img_file}' não está no formato correto e foi ignorado.")
                        
    return pd.DataFrame(data)

# Carregar o dataset
df = load_dataset_from_filenames(dataset_path)

# Verificar se o DataFrame foi carregado corretamente
if df.empty:
    print("Nenhum dado foi carregado. Verifique se as pastas e arquivos estão corretos.")
else:
    # Exibir a Tabela Sumária dos Dados
    print("Tabela Sumária dos Dados:")
    print(df.head())

    # Contar o número de classes e de imagens
    num_classes = df['classe'].nunique()
    num_imagens = df.shape[0]
    
    # Verificar se todos os arquivos têm caminhos válidos antes de calcular o tamanho total
    if not df['caminho'].isnull().values.any():
        total_size = sum(os.path.getsize(f) for f in df['caminho']) / (1024 * 1024)  # Convertido para MB
        print(f"Número de classes: {num_classes}")
        print(f"Número de imagens: {num_imagens}")
        print(f"Tamanho total da base: {total_size:.2f} MB")
    else:
        print("Alguns arquivos não têm caminhos válidos. Verifique o dataset.")
    
    # Salvar o DataFrame no Google Drive como CSV
    output_path = '/content/drive/My Drive/vision_uerj/dataset_metadata.csv'
    df.to_csv(output_path, index=False)
    print(f"Arquivo salvo em: {output_path}")
